In [17]:
import pandas as pd
import numpy as np
import os
import gradio as gr
from dotenv import load_dotenv
from operator import itemgetter



from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub
from langchain.schema import AIMessage, HumanMessage, SystemMessage




from langchain_core.runnables import (
    ConfigurableField,
    RunnableBinding,
    RunnableLambda,
    RunnablePassthrough,
    RunnableParallel
)

pd.set_option('display.max_colwidth', 300)  # Or use a large number if 'None' does not work in some environments
pd.set_option('display.max_columns', 300)  # Show all columns
pd.set_option('display.max_rows', 20)  # Show all row

load_dotenv()
OPENAI_APIKEY = os.environ['OPENAI_APIKEY']

In [38]:
model = ChatOpenAI(api_key=OPENAI_APIKEY, model='gpt-3.5-turbo', temperature=0.5)

In [16]:
# Try just the runnable parallel
chain = RunnableParallel({'question': RunnablePassthrough(), 'language': RunnablePassthrough()})

In [13]:
# both question and language and dictionaries...
chain.invoke({'question': "tell me a joke", 'language': 'english'})

{'question': {'question': 'tell me a joke', 'language': 'english'},
 'language': {'question': 'tell me a joke', 'language': 'english'}}

In [23]:
def get_int(x):
    return int(x)

In [31]:
# Use itemgetter or lambda functions to mainpulate the inputs
chain = RunnableParallel({'question': lambda x: x['question'] + '35', 'language': itemgetter('language'), 'number': lambda x: get_int(x['number'])})

In [32]:
# Now it works!
chain.invoke({'question': "tell me a joke", 'language': 'english', 'number': '8'})

{'question': 'tell me a joke35', 'language': 'english', 'number': 8}

In [35]:
# Don't have to use a dictionary if using RunnableParalle
chain = RunnableParallel(question = lambda x: x['question'] + "35",language = itemgetter('language'), number = lambda x: get_int(x['number']))

In [36]:
chain.invoke({'question': "tell me a joke", 'language': 'english', 'number': '8'})

{'question': 'tell me a joke35', 'language': 'english', 'number': 8}

In [39]:
# Run steps in parallel
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': AIMessage(content='Why did the bear bring a flashlight to the party? \n\nBecause he wanted to be the "light" of the party!', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 13, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-a1b6c849-3e21-4b38-8333-bc633bfc7f41-0'),
 'poem': AIMessage(content='In the forest deep and dark, a bear roams free\nMajestic and powerful, a sight to see', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 15, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-dc62bbe8-9537-4747-ae70-5addb13a6c4f-0')}

# Binding

In [43]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...EXPLANATION:...\n\n",
        ),
        ("human", "{equation_statement}"),
    ]
)
runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model.bind(stop='EXPLANATION') | StrOutputParser() # Add the bind to the model. What else could I bind?
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12
SOLUTION: x^3 = 12 - 7 = 5
x = ∛5
SOLUTION: x ≈ 1.71



# Functions

In [45]:
function = {
    "name": "solver",
    "description": "Formulates and solves an equation",
    "parameters": {
        "type": "object",
        "properties": {
            "equation": {
                "type": "string",
                "description": "The algebraic expression of the equation",
            },
            "solution": {
                "type": "string",
                "description": "The solution to the equation",
            },
        },
        "required": ["equation", "solution"],
    },
}

In [49]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
    
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [57]:
model2 = model.bind(tools=tools)
result = model2.invoke("What's your name?")

In [58]:
result.tool_calls

[]

In [61]:
result = model2.invoke("What's the weather in LA in farenheit")

In [62]:
result.tool_calls

[{'name': 'get_current_weather',
  'args': {'location': 'Los Angeles, CA', 'unit': 'fahrenheit'},
  'id': 'call_cyAmR39IrnBZq7i5OfGrhOvZ'}]

Function calling let's the AI optionally give you the inputs to a function if it makes sense from the prompt

In [ ]:
model.configurable_fields()